# Get AIrsenal's predictions for a set of fixtures

In [1]:
import numpy as np

from airsenal.framework.bpl_interface import get_fitted_team_model
from airsenal.framework.schema import session
from airsenal.framework.utils import get_fixtures_for_gameweek

In [2]:
season = "2122"
gw = 8
max_goals = 10

In [3]:
model = get_fitted_team_model(season, gw, session)

Fitting team model...


sample: 100%|█████████████████████████████████| 1500/1500 [00:22<00:00, 66.97it/s, 63 steps of size 6.12e-02. acc. prob=0.94]


In [4]:
fixtures = get_fixtures_for_gameweek(gw, season, session)

h_goals = []
a_goals = []
for h in range(max_goals):
    for a in range(max_goals):
        h_goals.append(h)
        a_goals.append(a)

In [5]:
score_probs = [
    model.predict_score_proba(
        f.home_team, f.away_team, np.array(h_goals), np.array(a_goals)
    )
    for f in fixtures
]
most_likely_idx = [sp.argmax() for sp in score_probs]
most_likely_score = [(h_goals[idx], a_goals[idx]) for idx in most_likely_idx]

In [6]:
outcome_probs = [
    model.predict_outcome_proba(f.home_team, f.away_team) for f in fixtures
]
most_likely_outcome = []
for o in outcome_probs:
    if (o["home_win"] >= o["away_win"]) and (o["home_win"] >= o["draw"]):
        most_likely_outcome.append("home_win")
    elif (o["away_win"] > o["home_win"]) and (o["away_win"] >= o["draw"]):
        most_likely_outcome.append("away_win")
    else:
        most_likely_outcome.append("draw")

In [7]:
goal_probs = [
    (
        model.predict_score_n_proba(
            np.arange(max_goals), f.home_team, f.away_team, home=True
        ),
        model.predict_score_n_proba(
            np.arange(max_goals), f.away_team, f.home_team, home=False
        ),
    )
    for f in fixtures
]
mean_goals = [
    (
        round(np.average(np.arange(max_goals), weights=g[0]), 2),
        round(np.average(np.arange(max_goals), weights=g[1]), 2),
    )
    for g in goal_probs
]

In [21]:
for f, s, o, m in zip(fixtures, most_likely_score, most_likely_outcome, mean_goals):
    if f.result:
        actual_prob = model.predict_score_proba(
            f.home_team, f.away_team, f.result.home_score, f.result.away_score
        )
        print(f.result, ", PREDICT:", o, s, m, f", ACTUAL: {float(actual_prob):.1%}")
    else:
        print(f, "pred", o, s, m)

2122 GW8 WAT 0 - 5 LIV , PREDICT: away_win (0, 2) (0.78, 2.38) , ACTUAL: 2.7%
2122 GW8 AVL 2 - 3 WOL , PREDICT: away_win (1, 1) (1.17, 1.22) , ACTUAL: 1.9%
2122 GW8 LEI 4 - 2 MUN , PREDICT: away_win (1, 1) (1.36, 1.43) , ACTUAL: 0.9%
2122 GW8 MCI 2 - 0 BUR , PREDICT: home_win (2, 0) (2.68, 0.65) , ACTUAL: 12.8%
2122 GW8 NOR 0 - 0 BHA , PREDICT: away_win (1, 1) (1.06, 1.25) , ACTUAL: 10.5%
2122 GW8 SOU 1 - 0 LEE , PREDICT: home_win (1, 1) (1.46, 1.32) , ACTUAL: 8.8%
2122 GW8 BRE 0 - 1 CHE , PREDICT: away_win (0, 2) (0.77, 2.16) , ACTUAL: 11.3%
2122 GW8 EVE 0 - 1 WHU , PREDICT: home_win (1, 1) (1.45, 1.21) , ACTUAL: 8.1%
2122 GW8 NEW 2 - 3 TOT , PREDICT: away_win (1, 1) (1.07, 1.75) , ACTUAL: 3.0%
2122 GW8 ARS 2 - 2 CRY , PREDICT: home_win (1, 0) (1.81, 0.93) , ACTUAL: 4.5%
